# Holder Wallet Conformance Flows (v3) - CTWalletSamePreAuthorised

# 0.0 Initial setup

## 0.1 Setup conformance

In [6]:
import uuid
import asyncio
from rich.console import Console

console = Console()

loop = asyncio.get_event_loop()

## 0.2 Create did:key:jwk_jcs-pub identifier using ES256 algorithm

In [7]:
from eudi_wallet.did_key import KeyDid, PublicKeyJWK
import uuid

# generate crypto seed
crypto_seed = b'ebsitests'

key_did = KeyDid(seed=crypto_seed)

# generate keypair
key_did.create_keypairEd25519()
print(key_did.public_key_jwk)

# create public key jwk
public_key_jwk = PublicKeyJWK(
    kty=key_did.public_key_jwk['kty'],
    crv=key_did.public_key_jwk['crv'],
    x=key_did.public_key_jwk['x'],
    # y=key_did.public_key_jwk['y'],
    kid=key_did.public_key_jwk['kid']
)

# generate did
key_did.generate_did(public_key_jwk)

print("Decentralised identifier: ", key_did._did)

{'kty': 'OKP', 'kid': 'Rh2DCux8Iwh4SHjJ-_n6GA9sxC8KIjgiIzCbPbfCCIc', 'crv': 'Ed25519', 'x': 'oD-iNX6LjLX3CPOBdihJ-13r4PEQsJ2DZtV-Q_LthN0'}
Decentralised identifier:  did:key:z3enk3Kcae5QHPRCd74LiiC9W9kjXNuXbuif4XiMrZh75qsYT311m3m2R1p6AH3CWhf55TLRFUxyjgGk7G6YzAtPd7Dm9QyZePN8nQeXYSFZZaPRfWsfH28STRMVYHv1xrwS43cicti5xRsRUyZh2MPeTtkVWq5d1yXp8qqsfdCpaVgri4Bt9boBY9e82vBPcmVfa


## 1.1 Initiate Credential Issuance

In [8]:
from eudi_wallet.util import parse_query_string_parameters_from_url
from eudi_wallet.siop_auth.util import (
    accept_and_fetch_credential_offer, 
    fetch_openid_credential_issuer_configuration,
    fetch_openid_auth_server_configuration,
    fetch_credential_offer,
    CredentialTypes
)

qr_code_data = "openid-credential-offer://?credential_offer_uri=https%3A%2F%2Fopenid-dts-dev.dev.adaptivespace.io%2Fcredential-offer%2F018f9b2f-5eef-79b6-a51a-2559097de503"
qr_code_data = qr_code_data.replace("openid-credential-offer://", "")
credential_offer_uri = parse_query_string_parameters_from_url(
    qr_code_data).get("credential_offer_uri")[0]
console.log("Credential offer URI: ", credential_offer_uri)

credential_offer = await accept_and_fetch_credential_offer(credential_offer_uri)
console.log("Credential offer: ", credential_offer)

credential_issuer_configuration = await fetch_openid_credential_issuer_configuration(credential_offer.credential_issuer)
console.log("Credential issuer configuration: ", credential_issuer_configuration)

auth_server_configuration = await fetch_openid_auth_server_configuration(credential_issuer_configuration.authorization_servers[0])
console.log("Authorization server configuration: ", auth_server_configuration)

[12:45:35] Credential offer URI:                                                                    ]8;id=599849;file:///tmp/ipykernel_47406/247062773.py\247062773.py]8;;\:]8;id=194388;file:///tmp/ipykernel_47406/247062773.py#14\14]8;;\
           https://openid-dts-dev.dev.adaptivespace.io/credential-offer/018f9b2f-5eef-79b6-a51a-255                
           9097de503                                                                                               

           Credential offer:                                                                        ]8;id=937784;file:///tmp/ipykernel_47406/247062773.py\247062773.py]8;;\:]8;id=359154;file:///tmp/ipykernel_47406/247062773.py#17\17]8;;\
           CredentialOffer(                                                                                        
               credential_issuer='https://openid-dts-dev.dev.adaptivespace.io/issuers/018f2a1b-7e08                
           -74c4-ac06-11497a90df72',                                                                               
               credentials=['testTemplateJWT2_JWT_VC_JSON'],                                                       
               grants={                                                                                            
                   'urn:ietf:params:oauth:grant-type:pre-authorized_code': {                                       
                       'pre-authorized_code': 'LaQ230jJSm8VGRdtuGeY1pHg41FegN67',                                  
                       'user_pin_required': False                                                                  
                   }                                                                                               
               }                                                                                                   
           )                                                                                                       

           Credential issuer configuration:                                                         ]8;id=887076;file:///tmp/ipykernel_47406/247062773.py\247062773.py]8;;\:]8;id=557091;file:///tmp/ipykernel_47406/247062773.py#20\20]8;;\
           OpenIDCredentialIssuerConfig(                                                                           
               credential_issuer='https://openid-dts-dev.dev.adaptivespace.io/issuers/018f2a1b-7e08                
           -74c4-ac06-11497a90df72',                                                                               
               authorization_servers=[                                                                             
                   'https://openid-dts-dev.dev.adaptivespace.io/issuers/018f2a1b-7e08-74c4-ac06-114                
           97a90df72'                                                                                              
               ],                                                                                                  
               token_endpoint='https://openid-dts-dev.dev.adaptivespace.io/token',                                 
               credential_endpoint='https://openid-dts-dev.dev.adaptivespace.io/credential',                       
               credentials_supported={                                                                             
                   'testTemplateJWT_JWT_VC_JSON': {                                                                
                       'format': 'jwt_vc_json',                                                                    
                       'cryptographic_binding_methods_supported': ['did', 'jwk'],                                  
                       'cryptographic_suites_supported': ['EdDSA'],                                                
                       'proof_types_supported': ['jwt'],                                                           
                       'credential_definition': {                                                                  
                           'type': ['VerifiableCredential', 'IATACredential']                                      
                       }                                                                                           
                   },                                                                                              
                   'testTemplateJWT2_JWT_VC_JSON': {                                                               
                       'format': 'jwt_vc_json',                                                                    
                       'cryptographic_binding_methods_supported': ['did', 'jwk'],                                  
                       'cryptographic_suites_supported': ['EdDSA'],                                                
                       'proof_types_supported': ['jwt'],                                                           
                       'credential_definition': {                                                                  
                           'type': ['VerifiableCredential', 'IATACredential']                                      
                       }                                                                                           
                   }                                                                                               
               },                                                                                                  
               deferred_credential_endpoint=None,                                                                  
               authorization_server=None,                                                                          
               display=[{'name': 'agent1'}]                                                                        
           )                               

           Authorization server configuration:                                                      ]8;id=48901;file:///tmp/ipykernel_47406/247062773.py\247062773.py]8;;\:]8;id=893780;file:///tmp/ipykernel_47406/247062773.py#23\23]8;;\
           OpenIDAuthServerConfig(                                                                                 
               redirect_uris=None,                                                                                 
               issuer='https://openid-dts-dev.dev.adaptivespace.io/issuers/018f2a1b-7e08-74c4-ac06-                
           11497a90df72',                                                                                          
               authorization_endpoint=None,                                                                        
               token_endpoint='https://openid-dts-dev.dev.adaptivespace.io/token',                                 
               jwks_uri=None,                                                                                      
               scopes_supported=None,                                                                              
               response_types_supported=None,                                                                      
               response_modes_supported=None,                                                                      
               grant_types_supported=None,                                                                         
               subject_types_supported=None,                                                                       
               id_token_signing_alg_values_supported=None,                                                         
               request_object_signing_alg_values_supported=None,                                                   
               request_parameter_supported=None,                                                                   
               request_uri_parameter_supported=None,                                                               
               token_endpoint_auth_methods_supported=None,                                                         
               request_authentication_methods_supported=None,                                                      
               vp_formats_supported=None,                                                                          
               subject_syntax_types_supported=None,                                                                
               subject_syntax_types_discriminations=None,                                                          
               subject_trust_frameworks_supported=None,                                                            
               id_token_types_supported=None                                                                       
           )                                                                                                       

## 1.2 Exchange code for access token

In [9]:
from eudi_wallet.siop_auth.util import exchange_pre_authorized_code_for_access_token

token_uri = auth_server_configuration.token_endpoint
access_token_response = await exchange_pre_authorized_code_for_access_token(token_uri, None, credential_offer.grants.get("urn:ietf:params:oauth:grant-type:pre-authorized_code").get("pre-authorized_code"))
console.log("Access token response: ", access_token_response)

           Access token response:                                                                   ]8;id=444066;file:///tmp/ipykernel_47406/1125731764.py\1125731764.py]8;;\:]8;id=547097;file:///tmp/ipykernel_47406/1125731764.py#5\5]8;;\
           AccessTokenResponse(                                                                                    
               access_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJleHAiOjE3MTYyOTY0MzUsImlhdCI6M                
           TcxNjI5NTUzNSwianRpIjoiNDgxNjA0OTgtZTI4Ni00OWRiLWE1MmMtMGY2MzQwNmE0YTJmIn0.Axtz8U_Sj8a58                
           R9GmU5KKcdLXLPkVatDMIajYs5mr8Y',                                                                        
               token_type='Bearer',                                                                                
               expires_in=None,                                                                                    
               id_token=None,                                                                                      
               c_nonce='Po9C8lzLOT',                                                                               
               c_nonce_expires_in=None,                                                                            
               scope=None                                                                                          
           )                                                                                                       

## 1.3 Request credential (same device)

In [10]:
from eudi_wallet.siop_auth.util import send_credential_request

credential_request_jwt = key_did.generate_credential_requestEd25519(credential_issuer_configuration.credential_issuer, access_token_response.c_nonce)
console.log("Credential request JWT: ", credential_request_jwt)
console.log("Credential types: ", credential_issuer_configuration.credentials_supported[credential_offer.credentials[0]]['credential_definition']['type'])
credential_response = await send_credential_request(credential_issuer_configuration.credential_endpoint, access_token_response.access_token, credential_request_jwt, credential_issuer_configuration.credentials_supported[credential_offer.credentials[0]]['credential_definition']['type'])
print("Credential response: ", credential_response)

           Credential request JWT:                                                                  ]8;id=954093;file:///tmp/ipykernel_47406/3746879363.py\3746879363.py]8;;\:]8;id=961940;file:///tmp/ipykernel_47406/3746879363.py#4\4]8;;\
           eyJhbGciOiJFZERTQSIsImtpZCI6Imh0dHBzOi8vb3BlbmlkLWR0cy1kZXYuZGV2LmFkYXB0aXZlc3BhY2UuaW8v                
           aXNzdWVycy8wMThmMmExYi03ZTA4LTc0YzQtYWMwNi0xMTQ5N2E5MGRmNzIjUmgyREN1eDhJd2g0U0hqSi1fbjZH                
           QTlzeEM4S0lqZ2lJekNiUGJmQ0NJYyIsInR5cCI6Im9wZW5pZDR2Y2ktcHJvb2Yrand0In0.eyJhdWQiOiJQbzlD                
           OGx6TE9UIiwiZXhwIjoxNzE2MzgxOTM1LCJpYXQiOjE3MTYyOTU1MzUsImlzcyI6ImRpZDprZXk6ejNlbmszS2Nh                
           ZTVRSFBSQ2Q3NExpaUM5VzlralhOdVhidWlmNFhpTXJaaDc1cXNZVDMxMW0zbTJSMXA2QUgzQ1doZjU1VExSRlV4                
           eWpnR2s3RzZZekF0UGQ3RG05UXlaZVBOOG5RZVhZU0ZaWmFQUmZXc2ZIMjhTVFJNVllIdjF4cndTNDNjaWN0aTV4                
           UnNSVXlaaDJNUGVUdGtWV3E1ZDF5WHA4cXFzZmRDcGFWZ3JpNEJ0OWJvQlk5ZTgydkJQY21WZmEiLCJub25jZSI6                
           bnVsbH0.pXGPJE9YaatUptY7-heFhMQiu8xU0MrKjiznCdjx7SUQPssfy2a2jB3kDm8rKpAzsinDJAuNbofPFeQf                
           Qq3yDA                                                                                                  

           Credential types:                                                                        ]8;id=330660;file:///tmp/ipykernel_47406/3746879363.py\3746879363.py]8;;\:]8;id=519115;file:///tmp/ipykernel_47406/3746879363.py#5\5]8;;\
           ['VerifiableCredential', 'IATACredential']                                                              

Credential response:  {'error': 'invalid_proof'}
